# Experiment 1: Command Interface Tuning
## Setup
- Launch analysis nodes
    - scene est
    - 

## Analysis

In [1]:
# Imports
import os

import numpy as np

import rosbag2_py
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message

import rclpy
from rclpy.node import Node

from situated_hri_interfaces.srv import SceneVisRec, SceneAudioRec

In [2]:
# Parameters
filepath = '../../bags/e1_est_tuning'

In [3]:
# Member variables
rclpy.init()
node = Node('analysis_script_node')
clip_scene_rec_client = node.create_client(SceneVisRec, 'scene_vis_rec_service')
audio_scene_rec_client = node.create_client(SceneAudioRec, 'scene_audio_rec_service')

# scene recognition models
actual_scene_labels = ['campus','courtyard','lab','lobby']

clip_scene_labels = ['college-campus','courtyard','laboratory','lobby']
clip_scene_model = np.zeros((len(actual_scene_labels),len(clip_scene_labels)))

cpjku_audio_scene_labels = ['airport', 'bus', 'metro', 'metro_station', 'park', 'public_square', 'shopping_mall', 'street_pedestrian', 'street_traffic', 'tram']
cpjku_audio_scene_model = np.zeros((len(actual_scene_labels),len(cpjku_audio_scene_labels)))

# Member functions
def typename(topic_name):
    for topic_type in topic_types:
        if topic_type.name == topic_name:
            return topic_type.type
    raise ValueError(f"topic {topic_name} not in bag")

In [4]:
# Traverse filepath
for root, dirs, files in os.walk(filepath):

    # Examine each file
    for file in files:
        
        # If it is an mcap, examine
        if os.path.splitext(file)[-1] == '.mcap':

            # Get labels based on filepath
            labels = os.path.normpath(root).split(os.sep)
            command_actual = labels[-1]
            modality_actual = labels[-2]
            role_actual = labels[-3]
            scene_actual = labels[-4]

            scene_actual_idx = actual_scene_labels.index(scene_actual)
            
            print('Command: %s\nModality: %s\nRole: %s\nScene: %s (%s)\n\n' % (command_actual, modality_actual, role_actual, scene_actual, scene_actual_idx))
            
            reader = rosbag2_py.SequentialReader()            
            reader.open(
                rosbag2_py.StorageOptions(uri=os.path.join(root,file), storage_id="mcap"),
                rosbag2_py.ConverterOptions(
                    input_serialization_format="cdr", output_serialization_format="cdr"
                ),
            )
            topic_types = reader.get_all_topics_and_types()

            # Iterate through messages
            while reader.has_next():
                topic, data, timestamp = reader.read_next()
                msg_type = get_message(typename(topic))

                # Handle audio scene data
                if topic=='/audio_data':
                    audio_scene_req = SceneAudioRec.Request()
                    audio_scene_req.scene_audio = deserialize_message(data,msg_type)
                    future = audio_scene_rec_client.call_async(audio_scene_req)
                    rclpy.spin_until_future_complete(node, future)
                    
                    est_scene_idx = np.argmax(future.result().scene_class.probabilities)
                    cpjku_audio_scene_model[scene_actual_idx,est_scene_idx] += 1
                
                # Handle visual scene data
                if topic=='/oak/rgb/image_raw':
                    clip_scene_img_req = SceneVisRec.Request()
                    clip_scene_img_req.scene_image = deserialize_message(data,msg_type)
                    future = clip_scene_rec_client.call_async(clip_scene_img_req)
                    rclpy.spin_until_future_complete(node, future)
                    
                    est_scene_idx = np.argmax(future.result().scene_class.probabilities)
                    clip_scene_model[scene_actual_idx,est_scene_idx] += 1

            print("CLIP SCENE OBSERVATION MODEL")
            print(clip_scene_model)
            print("\n")

            print("DPJKU AUDIO SCENE OBSERVATION MODEL")
            print(cpjku_audio_scene_model)
            print("\n")

            del reader

# Cleanup
rclpy.shutdown()

Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


CLIP SCENE OBSERVATION MODEL
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. 72. 25.]
 [ 0.  0.  0.  0.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


CLIP SCENE OBSERVATION MODEL
[[  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0. 151.  47.]
 [  0.   0.   0.   0.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 5. 1. 0. 0. 0. 0. 5. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


CLIP SCENE OBSERVATION MODEL
[[  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0. 243.  63.]
 [  0.   0.   0.   0.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 

[WARN] [1723075699.901780988] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


CLIP SCENE OBSERVATION MODEL
[[4.0920e+03 2.2000e+01 0.0000e+00 2.8680e+03]
 [4.1097e+04 1.7570e+03 0.0000e+00 5.3000e+01]
 [0.0000e+00 0.0000e+00 2.5793e+04 1.6399e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[5.000e+00 1.000e+00 1.200e+01 0.000e+00 2.900e+01 0.000e+00 0.000e+00
  0.000e+00 4.000e+00 3.690e+02]
 [2.000e+00 2.870e+02 1.000e+01 1.000e+01 7.950e+02 1.000e+00 0.000e+00
  1.000e+00 1.000e+01 1.349e+03]
 [1.000e+01 1.700e+02 2.109e+03 4.000e+01 8.000e+01 0.000e+00 0.000e+00
  0.000e+00 6.300e+01 2.500e+01]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]]


Command: move-in-reverse
Modality: verbal
Role: supervisor
Scene: campus (0)




[WARN] [1723075702.520853194] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


CLIP SCENE OBSERVATION MODEL
[[4.1010e+03 2.2000e+01 0.0000e+00 2.8680e+03]
 [4.1097e+04 1.7570e+03 0.0000e+00 5.3000e+01]
 [0.0000e+00 0.0000e+00 2.5793e+04 1.6399e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[5.000e+00 1.000e+00 1.200e+01 0.000e+00 2.900e+01 0.000e+00 0.000e+00
  0.000e+00 4.000e+00 3.690e+02]
 [2.000e+00 2.870e+02 1.000e+01 1.000e+01 7.950e+02 1.000e+00 0.000e+00
  1.000e+00 1.000e+01 1.349e+03]
 [1.000e+01 1.700e+02 2.109e+03 4.000e+01 8.000e+01 0.000e+00 0.000e+00
  0.000e+00 6.300e+01 2.500e+01]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]]


Command: move-in-reverse
Modality: verbal
Role: supervisor
Scene: campus (0)




...

CLIP SCENE OBSERVATION MODEL
[[25007.  5983.     0.  7662.]
 [41097.  1757.     0.    53.]
 [    0.     0. 25793. 16399.]
 [    0.     0.     0.  8297.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[1.400e+01 1.500e+01 4.100e+01 9.000e+00 1.950e+02 4.000e+00 1.000e+00
  1.000e+00 5.500e+01 1.943e+03]
 [2.000e+00 2.870e+02 1.000e+01 1.000e+01 7.950e+02 1.000e+00 0.000e+00
  1.000e+00 1.000e+01 1.349e+03]
 [1.000e+01 1.700e+02 2.109e+03 4.000e+01 8.000e+01 0.000e+00 0.000e+00
  0.000e+00 6.300e+01 2.500e+01]
 [1.500e+01 2.300e+01 4.400e+01 5.700e+01 2.650e+02 0.000e+00 6.000e+00
  0.000e+00 6.000e+00 7.000e+01]]


Command: rally
Modality: verbal
Role: supervisor
Scene: lobby (3)




[WARN] [1723078550.899777207] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


CLIP SCENE OBSERVATION MODEL
[[25007.  5983.     0.  7662.]
 [41097.  1757.     0.    53.]
 [    0.     0. 25793. 16399.]
 [    0.     0.     0.  8314.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[1.400e+01 1.500e+01 4.100e+01 9.000e+00 1.950e+02 4.000e+00 1.000e+00
  1.000e+00 5.500e+01 1.943e+03]
 [2.000e+00 2.870e+02 1.000e+01 1.000e+01 7.950e+02 1.000e+00 0.000e+00
  1.000e+00 1.000e+01 1.349e+03]
 [1.000e+01 1.700e+02 2.109e+03 4.000e+01 8.000e+01 0.000e+00 0.000e+00
  0.000e+00 6.300e+01 2.500e+01]
 [1.500e+01 2.300e+01 4.400e+01 5.700e+01 2.670e+02 0.000e+00 6.000e+00
  0.000e+00 6.000e+00 7.000e+01]]


Command: rally
Modality: verbal
Role: supervisor
Scene: lobby (3)


CLIP SCENE OBSERVATION MODEL
[[25007.  5983.     0.  7662.]
 [41097.  1757.     0.    53.]
 [    0.     0. 25793. 16399.]
 [    0.     0.     0.  8318.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[1.400e+01 1.500e+01 4.100e+01 9.000e+00 1.950e+02 4.000e+00 1.000e+00
  1.000e+00 5.500e+01 1.943e+03]
 [2.000e+00 2.870e+0

[WARN] [1723078551.647376773] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1723078551.977502463] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


CLIP SCENE OBSERVATION MODEL
[[25007.  5983.     0.  7662.]
 [41097.  1757.     0.    53.]
 [    0.     0. 25793. 16399.]
 [    0.     0.     0.  8332.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[1.400e+01 1.500e+01 4.100e+01 9.000e+00 1.950e+02 4.000e+00 1.000e+00
  1.000e+00 5.500e+01 1.943e+03]
 [2.000e+00 2.870e+02 1.000e+01 1.000e+01 7.950e+02 1.000e+00 0.000e+00
  1.000e+00 1.000e+01 1.349e+03]
 [1.000e+01 1.700e+02 2.109e+03 4.000e+01 8.000e+01 0.000e+00 0.000e+00
  0.000e+00 6.300e+01 2.500e+01]
 [1.500e+01 2.300e+01 4.500e+01 5.700e+01 2.680e+02 0.000e+00 6.000e+00
  0.000e+00 6.000e+00 7.000e+01]]


Command: halt
Modality: verbal
Role: supervisor
Scene: lobby (3)


CLIP SCENE OBSERVATION MODEL
[[25007.  5983.     0.  7662.]
 [41097.  1757.     0.    53.]
 [    0.     0. 25793. 16399.]
 [    0.     0.     0.  8417.]]


DPJKU AUDIO SCENE OBSERVATION MODEL
[[1.400e+01 1.500e+01 4.100e+01 9.000e+00 1.950e+02 4.000e+00 1.000e+00
  1.000e+00 5.500e+01 1.943e+03]
 [2.000e+00 2.870e+02